# Create JSON

[FOR JSON](https://docs.microsoft.com/en-us/sql/relational-databases/json/format-query-results-as-json-with-for-json-sql-server) allows you to turn a resultset into a _single_ JSON document:

```json
[
	{
		"firstName": "John",
		"lastName": "Smith",
		"isAlive": true,
		"age": 25,
		"address": [
			{
				"streetAddress": "21 2nd Street",
				"city": "New York",
				"state": "NY",
				"postalCode": "10021-3100"
			}
		]
	}
    ...
]
```

In [1]:
select 
	u.firstName,
	u.lastName,
	u.isAlive,
	u.age,
	[address].streetAddress,
	[address].city,
	[address].[state],
	[address].postalCode
from 
	dbo.users u 
inner join 
	dbo.user_addresses as [address] on u.id = [address].[user_id] 
for 
	json auto
go

(2 rows affected)

Total execution time: 00:00:00.014

JSON_F52E2B61-18A1-11d1-B105-00805F49916B
"[{""firstName"":""John"",""lastName"":""Smith"",""isAlive"":true,""age"":25,""address"":[{""streetAddress"":""21 2nd Street"",""city"":""New York"",""state"":""NY"",""postalCode"":""10021-3100""}]},{""firstName"":""Maggie"",""lastName"":""Doe"",""isAlive"":true,""age"":32,""address"":[{""streetAddress"":""109 1st Avenue"",""city"":""London"",""postalCode"":""SW1A0AA""}]}]"


With `JSON_QUERY` you can better decide how the nested JSON object is created:
```json
[
	{
		"firstName": "John",
		"lastName": "Smith",
		"isAlive": true,
		"age": 25,
		"address": {
			"streetAddress": "21 2nd Street",
			"city": "New York",
			"state": "NY",
			"postalCode": "10021-3100"
		}
    }
    ...
]
```

In [8]:
select 
	u.firstName,
	u.lastName,
	u.isAlive,
	u.age,
    json_query((select streetAddress, city, [state], postalCode from dbo.user_addresses ua where ua.[user_id] = u.id for json auto, without_array_wrapper)) as [address]
from 
	dbo.users u 
for 
	json auto
go

(2 rows affected)

Total execution time: 00:00:00.010

JSON_F52E2B61-18A1-11d1-B105-00805F49916B
"[{""firstName"":""John"",""lastName"":""Smith"",""isAlive"":true,""age"":25,""address"":{""streetAddress"":""21 2nd Street"",""city"":""New York"",""state"":""NY"",""postalCode"":""10021-3100""}},{""firstName"":""Maggie"",""lastName"":""Doe"",""isAlive"":true,""age"":32,""address"":{""streetAddress"":""109 1st Avenue"",""city"":""London"",""postalCode"":""SW1A0AA""}}]"


If you need even more control on how JSON is generated, you can use `FOR JSON PATH`. Columns alias will be used to create nested objects. For example: 
```json
[
	{
		"firstName": "John",
		"lastName": "Smith",
		"additionalInfo": {
			"isAlive": true,
			"age": 25
		}
	},
	...
]
```

In [4]:
select 
	u.firstName,
	u.lastName,    
	u.isAlive as 'additionalInfo.isAlive',
	u.age as 'additionalInfo.age'
from 
	dbo.users u 
for 
	json path
go


(2 rows affected)

Total execution time: 00:00:00.018

JSON_F52E2B61-18A1-11d1-B105-00805F49916B
"[{""firstName"":""John"",""lastName"":""Smith"",""additionalInfo"":{""isAlive"":true,""age"":25}},{""firstName"":""Maggie"",""lastName"":""Doe"",""additionalInfo"":{""isAlive"":true,""age"":32}}]"


## How to create one document per row?

If you want to **create one document per row**, instead of having one big JSON document with everything, you can use `JSON_QUERY` again:

In [11]:
select
    u2.id,
    json_query((
        select 
        u.firstName,
        u.lastName,
        u.isAlive,
        u.age,
        json_query((select streetAddress, city, [state], postalCode from dbo.user_addresses ua where ua.[user_id] = u.id for json auto, without_array_wrapper)) as [address]
        from 
            dbo.users u 
        where
            u.id = u2.id
        for 
            json auto
        )) as json_data
from
    dbo.users u2

(2 rows affected)

Total execution time: 00:00:00.034

id,json_data
1,"[{""firstName"":""John"",""lastName"":""Smith"",""isAlive"":true,""age"":25,""address"":{""streetAddress"":""21 2nd Street"",""city"":""New York"",""state"":""NY"",""postalCode"":""10021-3100""}}]"
2,"[{""firstName"":""Maggie"",""lastName"":""Doe"",""isAlive"":true,""age"":32,""address"":{""streetAddress"":""109 1st Avenue"",""city"":""London"",""postalCode"":""SW1A0AA""}}]"
